# Politics in Kenyon

In [1]:
SCHOOL="kenyon"
SUBJECT="opinion"
START_YEAR=2014
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [6]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [7]:
df.shape

(1072, 6)

In [8]:
df.head(2)

,title,url,body,year,month,day
18,"Column: At Kenyon, finding fiction is fictional",https://kenyoncollegian.com/opinion/2022/12/co...,"Column: At Kenyon, finding fiction is fictiona...",2022,12,8
569,Religious tradition extols nameless giving,https://kenyoncollegian.com/opinion/2017/09/re...,Religious tradition extols nameless giving\nTh...,2017,9,28


In [9]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [10]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [11]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

3.377299


In [12]:
df['year'].unique()

array([2022, 2017, 2016, 2015, 2023, 2021, 2020, 2019, 2018, 2014],
      dtype=int64)

In [13]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2014, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2014 ... 126
2014	126	56.002291	0.444462626984127
Year: 2015 ... 138
2015	138	-13.888597999999995	-0.10064201449275359
Year: 2016 ... 155
2016	155	-154.96300099999993	-0.9997612967741931
Year: 2017 ... 109
2017	109	-378.69566100000003	-3.4742721192660553
Year: 2018 ... 106
2018	106	-319.32265499999994	-3.01247787735849
Year: 2019 ... 96
2019	96	-241.16323099999994	-2.512116989583333
Year: 2020 ... 132
2020	132	-480.951146	-3.6435692878787878
Year: 2021 ... 100
2021	100	-359.2713930000001	-3.592713930000001
Year: 2022 ... 92
2022	92	-271.866383	-2.9550693804347823


In [14]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)